<a href="https://colab.research.google.com/github/abnerxch/bi-project/blob/main/ETL_de_Zips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adriana Mundo, Ábner Xocop y Fernanda González
# Epidemiología social del SARS-CoV-2: ETL de Zips

## Prep env

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import io
import requests
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Extract

In [ ]:
ZCTA_County_State = pd.read_csv('https://gist.githubusercontent.com/armi3/b8d197a5470e3165a968c547da9e9bd5/raw/1ef00d1476f0dde5217b6903e455881a9a5f5f57/ZCTA_County_State.csv')
ZCTA_County_State.info()
ZCTA_County_State.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42564 entries, 0 to 42563
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ZCTA    42564 non-null  int64 
 1   County  42564 non-null  object
 2   State   42564 non-null  object
dtypes: int64(1), object(2)
memory usage: 997.7+ KB


,ZCTA,County,State
0,35004,St. Clair AL,AL
1,35005,Jefferson AL,AL
2,35006,Jefferson AL,AL
3,35006,Tuscaloosa AL,AL
4,35007,Shelby AL,AL


In [ ]:
len(ZCTA_County_State.State.unique())

51

In [ ]:
zip_to_zcta_crosswalk_2020 = pd.read_csv('https://gist.githubusercontent.com/armi3/b8d197a5470e3165a968c547da9e9bd5/raw/e3aeef10ee5759c740fe5f72900d6d85cc49ecf0/zip_to_zcta_crosswalk_2020.csv')
zip_to_zcta_crosswalk_2020.info()
zip_to_zcta_crosswalk_2020.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41104 entries, 0 to 41103
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ZIP     41104 non-null  int64 
 1   ZCTA    41104 non-null  object
 2   STATE   41104 non-null  object
dtypes: int64(1), object(2)
memory usage: 963.5+ KB


,ZIP,ZCTA,STATE
0,501,11742,NY
1,544,11742,NY
2,601,00601,PR
3,602,00602,PR
4,603,00603,PR


In [ ]:
len(zip_to_zcta_crosswalk_2020.STATE.unique())
zip_to_zcta_crosswalk_2020.STATE.unique()

array(['NY', 'PR', 'VI', 'MA', 'RI', 'NH', 'ME', 'VT', 'CT', 'NJ', 'PA',
       'DE', 'DC', 'VA', 'MD', 'WV', 'NC', 'SC', 'GA', 'FL', 'AL', 'TN',
       'MS', 'KY', 'OH', 'IN', 'MI', 'IA', 'WI', 'MN', 'SD', 'ND', 'MT',
       'IL', 'MO', 'KS', 'NE', 'LA', 'AR', 'OK', 'TX', 'CO', 'WY', 'ID',
       'UT', 'AZ', 'NM', 'NV', 'CA', 'HI', 'AS', 'GU', 'PW', 'FM', 'MP',
       'MH', 'OR', 'WA', 'AK'], dtype=object)

## Transform

In [ ]:
zip_to_zcta_crosswalk_2020['ZCTA'] = pd.to_numeric(zip_to_zcta_crosswalk_2020['ZCTA'].replace('No ZCTA', np.nan))
zip_to_zcta_crosswalk_2020 = zip_to_zcta_crosswalk_2020[['ZIP','ZCTA']]
zip_to_zcta_crosswalk_2020.info()
zip_to_zcta_crosswalk_2020.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41104 entries, 0 to 41103
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ZIP     41104 non-null  int64  
 1   ZCTA    41096 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 642.4 KB


,ZIP,ZCTA
0,501,11742.0
1,544,11742.0
2,601,601.0
3,602,602.0
4,603,603.0


In [ ]:
zip_to_zcta = pd.merge(zip_to_zcta_crosswalk_2020, ZCTA_County_State, on="ZCTA")
us_state_abbrev = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AS": "American Samoa",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "DC": "District Of Columbia",
    "FM": "Federated States Of Micronesia",
    "FL": "Florida",
    "GA": "Georgia",
    "GU": "Guam",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MH": "Marshall Islands",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "MP": "Northern Mariana Islands",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PW": "Palau",
    "PA": "Pennsylvania",
    "PR": "Puerto Rico",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VI": "Virgin Islands",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming"
}

zip_to_zcta['StateName'] = zip_to_zcta['State'].map(us_state_abbrev)
zip_to_zcta.head()

,ZIP,ZCTA,County,State,StateName
0,501,11742.0,Suffolk NY,NY,New York
1,544,11742.0,Suffolk NY,NY,New York
2,11742,11742.0,Suffolk NY,NY,New York
3,1001,1001.0,Hampden MA,MA,Massachusetts
4,1002,1002.0,Hampshire MA,MA,Massachusetts


In [ ]:
len(zip_to_zcta.State.unique())

51

## Load

In [ ]:
#result.to_csv('/content/drive/My Drive/datasources/ZIPs.csv', index = False)
zip_to_zcta.to_csv('/content/drive/My Drive/Documentos/2021 1/5. BI/Proyecto final/datasources/ZIPs.csv', index = False)